In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import time

2023-05-31 04:24:38.750874: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset, info = tfds.load('cifar10', shuffle_files=True, with_info=True, as_supervised=True)
train_dataset = dataset['train']
test_dataset = dataset['test']

train_dataset = train_dataset.map(lambda image, label: (tf.image.resize(image, (32, 32)), label))
test_dataset = test_dataset.map(lambda image, label: (tf.image.resize(image, (32, 32)), label))

train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32)

2023-05-31 04:24:40.607455: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-31 04:24:40.607484: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: b95838f43c49
2023-05-31 04:24:40.607491: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: b95838f43c49
2023-05-31 04:24:40.607534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 530.30.2
2023-05-31 04:24:40.607552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.30.2
2023-05-31 04:24:40.607558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 530.30.2


In [3]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10))

In [4]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
start_time = time.time()
with tf.device('/GPU:0'):
    model.fit(train_dataset, epochs=10, validation_data=test_dataset)
end_time = time.time()
gpu_training_time = end_time - start_time

# Оценка точности модели на GPU
with tf.device('/GPU:0'):
    test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print('Accuracy of the network on the test images (GPU):', test_acc)
print('Training time on GPU:', gpu_training_time)

Epoch 1/10


2023-05-31 04:24:40.830807: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]
2023-05-31 04:24:40.831605: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]


1561/1563 [============================>.] - ETA: 0s - loss: 3.6658 - accuracy: 0.2070

2023-05-31 04:24:50.194921: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-05-31 04:24:50.195707: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]


1563/1563 [==============================] - 10s 6ms/step - loss: 3.6641 - accuracy: 0.2072 - val_loss: 2.1332 - val_accuracy: 0.2615
Epoch 2/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.9867 - accuracy: 0.2951 - val_loss: 2.3828 - val_accuracy: 0.3081
Epoch 3/10
1558/1563 [============================>.] - ETA: 0s - loss: 1.8487 - accuracy: 0.3457